# Task 2 - Find negative values

Given a list of integer numbers, look for a negative number in the list. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case

Example:
```python
A = find_negative_numbers([1,-3,2,15])
print(A)
```
```bash
"True"
```


```python
B = find_negative_numbers([1,4,8,11])
print(B)
```
```bash
"False"
```


```python
C = find_negative_numbers([-15,-14,2,-1])
print(C)
```
```bash
"True"
```

References:
1. Deutsch, David, and Richard Jozsa. "Rapid solution of problems by quantum computation." Proceedings of the Royal Society of London. Series A: Mathematical and Physical Sciences 439.1907 (1992): 553-558.
2. Bernstein, Ethan, and Umesh Vazirani. "Quantum complexity theory." SIAM Journal on computing 26.5 (1997): 1411-1473.
3. Grover, Lov K. , "A fast quantum mechanical algorithm for database search", Proceedings of the 28th Annual ACM Symposium on the Theory of Computing (1996), arXiv:quant-ph/9605043

In [7]:
import math
import numpy as np
import threading

from qiskit import QuantumRegister, QuantumCircuit
from qiskit.primitives import Sampler as AerSampler
from qiskit.quantum_info import Operator
from qiskit.visualization import plot_histogram

In [8]:
def construct_oracle_operator(signs, n):
    """
    'signs': list of signs (1, -1, or 0) which are to be input into the oracle

    Computes an oracle matrix using the formulation: `U_omega = I - 2|w><w|`,
    where `w` are the indices for which our search function `f(x) = (x < 0) = 1`,
    and returns a Qiskit Operator object created using the matrix.
    """

    # Initialize oracle matrix as an identity matrix
    oracle_matrix = np.eye(2**n)

    # Mark the first negative state by setting its matrix element equal to -1
    # We only mark the first occurrence because this amplifies its significance
    if -1 in signs:
        s = signs.index(-1)
        oracle_matrix[s][s] = -1

    # Create an oracle Operator from the oracle matrix
    oracle_operator = Operator(oracle_matrix)

    return oracle_operator


In [9]:

def construct_grover_diffusion_operator(n):
    """
    'list_len': length of the input list which is to be searched

    Computes a Grover diffusion matrix using the formulation: `U_s = 2|s><s| - I`,
    where `s` are the indices for which our search function `f(x) = (x < 0) = 1`,
    and returns a Qiskit Operator object created using the matrix.
    """

    # Initialize Grover matrix
    grover_diffusion_matrix = 2/(2 ** n) * np.ones(2**n) - np.eye(2**n)

    # Create a Grover diffusion Operator from the Grover diffusion matrix
    grover_diffusion_operator = Operator(grover_diffusion_matrix)

    return grover_diffusion_operator


In [10]:

def find_negative_numbers(list_number: list[int]):
    """
    `list_number`: list of integers to check

    Returns true if `list_number` contains a negative number, else false.
    """

    # We reduce each value to its sign (either 1, -1, or 0)
    signs = [int(number/np.abs(number)) if number != 0 else 0 for number in list_number]

    # Forcing the length of our list to be greater than or equal to 256 helps
    # amplify the significance of lists with fewer negative numbers
    scale_to = 256
    signs = signs * max(int(math.ceil(scale_to/len(signs))), 1)

    # Grover's algorithm requires a quantum register with `n = ceil(log_2(N))` qubits,
    # where N is the size of our search space (the inputs)
    N = len(signs)
    n = int(math.ceil(math.log2(N)))
    q_register = QuantumRegister(n)

    # Construct the oracle and Grover diffusion operators for our input
    oracle_operator = construct_oracle_operator(signs, n)
    grover_diffusion_operator = construct_grover_diffusion_operator(n)

    # Initialize a quantum circuit
    quantum_circuit = QuantumCircuit(q_register)
    # Shift to Hadamard basis
    quantum_circuit.h(q_register)

    # Maximum number of iterations for Grover's algorithm is around pi/4 * sqrt(N/M),
    # where M is the number of solutions - we assume the worst-case scenario where
    # M = 1, ensuring that this circuit works for any list
    rN = math.ceil(math.pi/4 * (N ** 0.5))
    for _ in range(rN):
        # Apply the oracle operator
        quantum_circuit.append(oracle_operator, q_register)

        # Apply the Grover diffusion operator
        quantum_circuit.append(grover_diffusion_operator, q_register)

    # Finally, measure all of the qubits in the QuantumRegister,
    # storing all measurements in a new ClassicalRegister
    quantum_circuit.measure_all(q_register)

    # Use Aer sampler to run 1024 shots of the circuit and get the result
    sampler = AerSampler(options={"shots": 1024})
    result = sampler.run(quantum_circuit).result()

    # Retrieve and sort quasi-probability list (can have negative entries)
    quasi_probs = list(result.quasi_dists[0].values())
    quasi_probs_sorted = sorted(quasi_probs, reverse=True)

    # We use the range as a significance metric because we expect outliers,
    # which skew standard deviation-based metrics in the expected case of a
    # true positive (a negative number, in this case)
    # In particular, we force more than an order of magnitude of range
    # between the max and min quasi-probabilities
    c = 11
    decision = (quasi_probs_sorted[0] == 1.0) or (quasi_probs_sorted[0] - c * quasi_probs_sorted[-1] > 0)

    # Classical method for checking if a negative number exists
    correct_answer = -1 in signs

    # Generate and save visualizations
    quantum_circuit.draw("mpl", filename="./qosf23_fnn_circuit_diagram.png")
    graph = plot_histogram(result.quasi_dists[0])
    graph.savefig("./qosf23_fnn_circuit_quasi_dist.png")

    # Return decision (True or False) to the main process
    return decision, correct_answer


In [11]:
# Try sample test cases
sample_tests = {
    "A": [1, -3, 2, 15],
    "B": [1, 4, 8, 11],
    "C": [-15, -14, 2, -1],
}

for sample_test_name, sample_test_list in sample_tests.items():
    guess, correct_answer = find_negative_numbers(sample_test_list)

    print(f"{'passed' if guess == correct_answer else 'did not pass'} sample test {sample_test_name} ({correct_answer})")


passed sample test A (True)
passed sample test B (False)
passed sample test C (True)


In [12]:
# Performing mass testing to ensure no boundary conditions are missed
test_lists = []

# Generates random tests for all lengths in the range [0,test_list_count)
test_list_count = 100
for length in range(2, 2 + test_list_count):
    test_lists.append(np.random.randint(-10, 100, length))

# Variables used to calculate accuracy later
correct_guesses = 0
total_guesses = 0

# Loop over test_lists
for t in range(test_list_count):
    test_list = test_lists[t]

    # Make a guess
    guess, correct_answer = find_negative_numbers(test_list)

    # Check if guess is correct
    if guess == correct_answer:
        correct_guesses += 1
        print(f"passed test list #{t} ({correct_answer})")
    else:
        print(f"did not pass test list #{t} ({correct_answer})")

    total_guesses += 1

# Calculate and print accuracy
accuracy = np.round(correct_guesses / total_guesses * 100, 2)
print(f"passed {correct_guesses}/{total_guesses} test cases ({accuracy}% accuracy)")

passed test list #0 (False)
passed test list #1 (True)
passed test list #2 (False)
passed test list #3 (True)
passed test list #4 (True)
passed test list #5 (True)
passed test list #6 (False)
passed test list #7 (True)
passed test list #8 (False)
passed test list #9 (True)
passed test list #10 (True)
passed test list #11 (True)
passed test list #12 (True)
passed test list #13 (False)
passed test list #14 (True)
passed test list #15 (False)
passed test list #16 (True)
passed test list #17 (True)
passed test list #18 (True)
passed test list #19 (False)
passed test list #20 (True)
passed test list #21 (True)
passed test list #22 (True)
passed test list #23 (True)
passed test list #24 (True)
passed test list #25 (True)
passed test list #26 (False)
passed test list #27 (True)
passed test list #28 (True)
passed test list #29 (True)
passed test list #30 (True)
passed test list #31 (True)
passed test list #32 (False)
passed test list #33 (True)
passed test list #34 (True)
passed test list #35 